In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import cupy as cp
import torch
import cv2
import os
import json
import plotly.graph_objects as go

from data_process import feature_map

In [2]:
with open("../cfg/cfg.json") as f:
    cfg = json.load(f)
    
dataset_dir = cfg['dataset_dir']

image_dir = os.path.join(dataset_dir, "training", "image_2")
lidar_dir = os.path.join(dataset_dir, "training", "velodyne")
calib_dir = os.path.join(dataset_dir, "training", "calib")
label_dir = os.path.join(dataset_dir, "training", "label_2")

In [3]:
def get_lidar(idx):
    lidar_file = os.path.join(lidar_dir, f"{idx:06d}.bin")
    return np.fromfile(lidar_file, dtype=np.float32).reshape(-1, 4)

def get_image(idx):
    img_path = os.path.join(image_dir, f'{idx:06d}.png')
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

    return img_path, img

def get_calib(idx):
    calib_file = os.path.join(calib_dir, f'{idx:06d}.txt')
    # assert os.path.isfile(calib_file)
    return Calibration(calib_file)

CLASS_NAME_TO_ID = {
    'Pedestrian': 0,
    'Car': 1,
    'Cyclist': 2,
    'Van': 1,
    'Truck': -3,
    'Person_sitting': 0,
    'Tram': -99,
    'Misc': -99,
    'DontCare': -1
}

def get_label(idx):
    labels = []
    label_path = os.path.join(label_dir, f'{idx:06d}.txt')
    for line in open(label_path, 'r'):
        line = line.rstrip()
        line_parts = line.split(' ')
        obj_name = line_parts[0]  # 'Car', 'Pedestrian', ...
        cat_id = int(CLASS_NAME_TO_ID[obj_name])
        if cat_id <= -99:  # ignore Tram and Misc
            continue
        '''
        truncated = int(float(line_parts[1]))  # truncated pixel ratio [0..1]
        occluded = int(line_parts[2])  # 0=visible, 1=partly occluded, 2=fully occluded, 3=unknown
        alpha = float(line_parts[3])  # object observation angle [-pi..pi]
        # xmin, ymin, xmax, ymax
        bbox = np.array([float(line_parts[4]), float(line_parts[5]), float(line_parts[6]), float(line_parts[7])])
        '''

        # height, width, length (h, w, l)
        h, w, l = float(line_parts[8]), float(line_parts[9]), float(line_parts[10])
        # location (x,y,z) in camera coord.
        x, y, z = float(line_parts[11]), float(line_parts[12]), float(line_parts[13])
        ry = float(line_parts[14])  # yaw angle (around Y-axis in camera coordinates) [-pi..pi]

        object_label = [cat_id, x, y, z, h, w, l, ry]
        labels.append(object_label)

    if len(labels) == 0:
        labels = np.zeros((1, 8), dtype=np.float32)
        has_labels = False
    else:
        labels = np.array(labels, dtype=np.float32)
        has_labels = True

    return labels, has_labels

In [4]:
f_idx = 7
point_cloud = get_lidar(f_idx)
label = get_label(f_idx)

In [5]:
frm = feature_map.frame(point_cloud, cfg)

Function 'get_base' executed in 0.0168s


In [6]:
frm.base

-1.75

### Read kernel code

In [ ]:
with open("grid_kernel.cu") as f:
    kernel_code = f.read()

In [ ]:
kernel_code

In [ ]:
with open("grid_kernel.cu") as f:
    kernel_code = f.read()

module = cp.RawModule(code=kernel_code, options=('--std=c++11',))
compute_grid_info = module.get_function('compute_grid_info')

In [ ]:
pos = point_cloud[point_cloud]

In [ ]:
frm.set_bev_map()

In [ ]:
idx_x = 0.5 * (frm.pos[:, 0] + cfg["x_range"]) * cfg["NX"] / cfg["x_range"]

In [ ]:
idx_x.int()